<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/scoring_lpips_lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring
This notebook takes the output test restults from the models and calculates their frechet inception distance

In [1]:
!git clone https://github.com/richzhang/PerceptualSimilarity
!pip install -r PerceptualSimilarity/requirements.txt

fatal: destination path 'PerceptualSimilarity' already exists and is not an empty directory.


In [2]:
import shutil
import os
import glob
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
test_output_lama = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/lama.zip'
test_output_ce = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ce.zip'
test_output_gal = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/gal.zip'
test_output_ffc = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ffc.zip'
test_dataset = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test.zip'

temp_dir_1 = '/content/temp_folder_1/'
temp_dir_2 = '/content/temp_folder_2/'

In [35]:
def unzip_images(input_zip):
    print(input_zip)

    print('Removing old folder')
    if os.path.isdir(temp_dir_1):
        shutil.rmtree(temp_dir_1)

    print('Unpacking zip file')
    shutil.copy(input_zip, '/content/')
    shutil.unpack_archive(f'/content/{os.path.basename(input_zip)}', temp_dir_1)

    file_list = glob.glob(temp_dir_1 + '*')
    print(f'Number of images: {len(file_list)}')

    # unpacks the test imagse if they are not already there
    if os.path.isdir(temp_dir_2) == False:
        shutil.copy(test_dataset, '/content/')
        shutil.unpack_archive(f'/content/{os.path.basename(test_dataset)}', temp_dir_2)
    

def rename_files(need_leading_zero=True):
    # rename the files
    temp_1_list = glob.glob(f'{temp_dir_1}*')
    temp_2_list = glob.glob(f'{temp_dir_2}*')

    temp_1_list.sort()
    temp_2_list.sort()

    print('Renaming files')

    if need_leading_zero:
        # have to copy and rename the files
        for i in range(len(temp_1_list)):
            new_file_name = os.path.basename(temp_1_list[i])
            new_file_name = new_file_name.split('_')[1]
            n_digits = len(new_file_name) - 4
            new_file_name = '0' * (5 - n_digits) + new_file_name

            # rename the file
            os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')    

    else:
        # have to copy and rename the files
        for i in range(len(temp_1_list)):
            new_file_name = str(i) + '.png'
            n_digits = len(new_file_name) - 4
            new_file_name = '0' * (5 - n_digits) + new_file_name

            # rename the file
            os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')

    # have to copy and rename test files
    for i in range(len(temp_2_list)):
        new_file_name = str(i) + '.jpg'
        n_digits = len(new_file_name) - 4
        new_file_name = '0' * (5 - n_digits) + new_file_name

        # rename the file
        os.rename(temp_2_list[i], f'{temp_dir_2}{new_file_name}')


def get_score():

    print('Calculating score')
    os.system('python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt --use_gpu')

    df = pd.read_csv('/content/PerceptualSimilarity/imgs/example_dists.txt', sep=' ', header=None)
    print(df.iloc[:, 1].mean())


In [21]:
# shutil.rmtree(temp_dir_2)
shutil.copy(test_dataset, '/content/')
shutil.unpack_archive(f'/content/{os.path.basename(test_dataset)}', temp_dir_2)

# files = glob.glob('/content/temp_folder_1/*')

In [22]:
# for z in [test_output_lama, test_output_ce, test_output_gal, test_output_ffc]:
unzip_images(test_output_lama)
# rename_files(False)

/content/gdrive/MyDrive/repos/GANime/data_out/test_output/lama.zip
Removing old folder
Unpacking zip file
Number of images: 61426


In [28]:
shutil.rmtree('/content/temp_folder_1/')

In [29]:
import cv2

file_list = glob.glob('/content/temp_folder_1_old/*')
file_list.sort()
new_dir = '/content/temp_folder_1/'

# have to copy and rename the files
for i in range(len(file_list)):
    new_file_name = str(i) + '.jpg'
    n_digits = len(new_file_name) - 4
    new_file_name = '0' * (5 - n_digits) + new_file_name

    img = cv2.imread(file_list[i])
    cv2.imwrite(new_dir + new_file_name, img)

    if i % 1500 == 0:
        print(i)

    # # rename the file
    # os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')

58500
60000


In [33]:
# rename the files
temp_2_list = glob.glob(f'{temp_dir_2}*')

temp_2_list.sort()

print('Renaming files')

# have to copy and rename test files
for i in range(len(temp_2_list)):
    new_file_name = str(i) + '.jpg'
    n_digits = len(new_file_name) - 4
    new_file_name = '0' * (5 - n_digits) + new_file_name

    # rename the file
    os.rename(temp_2_list[i], f'{temp_dir_2}{new_file_name}')

Renaming files


In [38]:
!python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt --use_gpu

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /content/PerceptualSimilarity/lpips/weights/v0.1/alex.pth
Traceback (most recent call last):
  File "PerceptualSimilarity/lpips_2dirs.py", line 17, in <module>
    loss_fn.cuda()
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 680, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 570, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 616, in _apply
    self._buffers[key] = fn(buf)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 680, in <lambda>
    return self._apply(lambda t: t.cuda(device))
  File "/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py", line 214, in _lazy_init
    torch._C._cuda_init()
RuntimeError: No CUDA GPUs are available
